In [1]:
from client import start_client
import matplotlib.pyplot as plt
import numpy as np
import time
from matplotlib.pyplot import figure
from main_func import main,real_partial
from plotting import plot, plot_workers, plot_speedup
from jupyterthemes import jtplot

client,w=start_client(33)
client

<Client: 'tcp://127.0.0.1:35933' processes=33 threads=33, memory=33.69 GB>


Client Scheduler: tcp://127.0.0.1:35933 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 33 Cores: 33 Memory: 33.69 GB


### Main function
- make_classification parameters: https://scikit-learn.org/stable/modules/generated/sklearn.datasets.make_classification.html?highlight=make_classification#sklearn.datasets.make_classification
- choose e, and chunk size
- run multiple times to produce plots


### Initialize
***

In [2]:
jtplot.style(theme='grade3') #light plot theme

batches=30000 #number of minibatches
batch_size=1 #size of each minibatch
e=0.8 #tolerance for φ(χ)

#compute samples size
training_s=batches*batch_size
n_samples=int(3*training_s/2) # 1/3 of the samples will be used for testing

#parameters for make classification
dataset_params={
    "n_samples":n_samples,
    "n_features":1000,
    "n_informative":10,
    "n_redundant":0,
    "n_repeated":0,
    "n_classes":2,
    "n_clusters_per_class":1,
    "flip_y":0,
    "weights":[0.7,0.3],
    "random_state":17
}
threshold=0.03

chunks=100
e_array=[0.3]
sizes=[30000,50000,100000,150000,200000,300000]
minibatches=[10,50,100]
new="no" #choose if you want to recreate the dataset

### Dataset with no noise flip_y=0
***
#### Balanced
- weights=[0.5,0,5]

In [ ]:
# 3 different e
new="no"
dataset_params["weights"]=[0.5,0.5]
Acc_threshold=[]
Rounds_threshold=[]
Time_threshold=[]
labels=[[]]

for i in e_array :
    e=i
    Acc,time_l,total_rounds,total_time=main(client,w,new,dataset_params,e,chunks,minibatches[0])
    new="no"
    Acc_threshold.append(Acc)
    Rounds_threshold.append(total_rounds)
    Time_threshold.append(time_l)
    time.sleep(5)

In [3]:
# centr=client.submit(real_partial,minibatches[1],workers=w[0])
# time_centr,Acc_real1,f_acc=centr.result()
time_centr,Acc_real1,f_acc=real_partial(10)
len_real1=np.array([i for i in range(len(Acc_real1))])
labels='centralized'
np.save('np_arrays/time_centr',time_centr)
np.save('np_arrays/Acc_real1',Acc_real1)
np.save('np_arrays/f_acc',f_acc)

Start...
----------------------------------------------

chunk 1
Minibaches 0
Accuracy: 74.846667
Minibaches 1
Accuracy: 75.793333
Minibaches 2
Accuracy: 75.633333
Minibaches 3
Accuracy: 75.866667
Minibaches 4
Accuracy: 76.873333
Minibaches 5
Accuracy: 77.006667
Minibaches 6
Accuracy: 78.066667
Minibaches 7
Accuracy: 81.013333
Minibaches 8
Accuracy: 81.406667
Minibaches 9
Accuracy: 82.440000
chunk 2
Minibaches 0
Accuracy: 81.820000
Minibaches 1
Accuracy: 83.453333
Minibaches 2
Accuracy: 84.053333
Minibaches 3
Accuracy: 84.526667
Minibaches 4
Accuracy: 84.306667
Minibaches 5
Accuracy: 84.013333
Minibaches 6
Accuracy: 84.140000
Minibaches 7
Accuracy: 84.400000
Minibaches 8
Accuracy: 84.066667
Minibaches 9
Accuracy: 84.066667
chunk 3
Minibaches 0
Accuracy: 84.093333
Minibaches 1
Accuracy: 85.146667
Minibaches 2
Accuracy: 84.653333
Minibaches 3
Accuracy: 84.960000
Minibaches 4
Accuracy: 84.780000
Minibaches 5
Accuracy: 85.240000
Minibaches 6
Accuracy: 85.160000
Minibaches 7
Accuracy: 85.29

Accuracy: 90.040000
Minibaches 8
Accuracy: 90.126667
Minibaches 9
Accuracy: 90.126667
chunk 26
Minibaches 0
Accuracy: 90.080000
Minibaches 1
Accuracy: 90.113333
Minibaches 2
Accuracy: 89.840000
Minibaches 3
Accuracy: 89.833333
Minibaches 4
Accuracy: 89.926667
Minibaches 5
Accuracy: 89.966667
Minibaches 6
Accuracy: 89.966667
Minibaches 7
Accuracy: 90.060000
Minibaches 8
Accuracy: 90.246667
Minibaches 9
Accuracy: 89.940000
chunk 27
Minibaches 0
Accuracy: 89.813333
Minibaches 1
Accuracy: 89.886667
Minibaches 2
Accuracy: 89.960000
Minibaches 3
Accuracy: 89.906667
Minibaches 4
Accuracy: 90.053333
Minibaches 5
Accuracy: 89.946667
Minibaches 6
Accuracy: 90.000000
Minibaches 7
Accuracy: 89.920000
Minibaches 8
Accuracy: 90.060000
Minibaches 9
Accuracy: 90.100000
chunk 28
Minibaches 0
Accuracy: 90.220000
Minibaches 1
Accuracy: 90.246667
Minibaches 2
Accuracy: 90.240000
Minibaches 3
Accuracy: 90.260000
Minibaches 4
Accuracy: 90.300000
Minibaches 5
Accuracy: 90.240000
Minibaches 6
Accuracy: 90.286

Accuracy: 91.526667
chunk 50
Minibaches 0
Accuracy: 91.493333
Minibaches 1
Accuracy: 91.613333
Minibaches 2
Accuracy: 91.513333
Minibaches 3
Accuracy: 91.360000
Minibaches 4
Accuracy: 91.433333
Minibaches 5
Accuracy: 91.446667
Minibaches 6
Accuracy: 91.533333
Minibaches 7
Accuracy: 91.506667
Minibaches 8
Accuracy: 91.440000
Minibaches 9
Accuracy: 91.600000
chunk 51
Minibaches 0
Accuracy: 91.600000
Minibaches 1
Accuracy: 91.560000
Minibaches 2
Accuracy: 91.473333
Minibaches 3
Accuracy: 91.406667
Minibaches 4
Accuracy: 91.513333
Minibaches 5
Accuracy: 91.600000
Minibaches 6
Accuracy: 91.593333
Minibaches 7
Accuracy: 91.560000
Minibaches 8
Accuracy: 91.613333
Minibaches 9
Accuracy: 91.473333
chunk 52
Minibaches 0
Accuracy: 91.633333
Minibaches 1
Accuracy: 91.613333
Minibaches 2
Accuracy: 91.640000
Minibaches 3
Accuracy: 91.700000
Minibaches 4
Accuracy: 91.560000
Minibaches 5
Accuracy: 91.480000
Minibaches 6
Accuracy: 91.240000
Minibaches 7
Accuracy: 91.246667
Minibaches 8
Accuracy: 91.380

Accuracy: 92.053333
Minibaches 8
Accuracy: 91.973333
Minibaches 9
Accuracy: 92.040000
chunk 75
Minibaches 0
Accuracy: 91.886667
Minibaches 1
Accuracy: 91.886667
Minibaches 2
Accuracy: 91.893333
Minibaches 3
Accuracy: 91.840000
Minibaches 4
Accuracy: 91.866667
Minibaches 5
Accuracy: 91.806667
Minibaches 6
Accuracy: 91.806667
Minibaches 7
Accuracy: 91.806667
Minibaches 8
Accuracy: 91.793333
Minibaches 9
Accuracy: 91.880000
chunk 76
Minibaches 0
Accuracy: 91.800000
Minibaches 1
Accuracy: 91.873333
Minibaches 2
Accuracy: 91.880000
Minibaches 3
Accuracy: 91.880000
Minibaches 4
Accuracy: 91.886667
Minibaches 5
Accuracy: 91.853333
Minibaches 6
Accuracy: 91.833333
Minibaches 7
Accuracy: 91.806667
Minibaches 8
Accuracy: 91.760000
Minibaches 9
Accuracy: 91.773333
chunk 77
Minibaches 0
Accuracy: 91.906667
Minibaches 1
Accuracy: 91.920000
Minibaches 2
Accuracy: 92.000000
Minibaches 3
Accuracy: 91.793333
Minibaches 4
Accuracy: 91.840000
Minibaches 5
Accuracy: 91.840000
Minibaches 6
Accuracy: 91.893

Accuracy: 92.546667
Minibaches 2
Accuracy: 92.506667
Minibaches 3
Accuracy: 92.506667
Minibaches 4
Accuracy: 92.513333
Minibaches 5
Accuracy: 92.526667
Minibaches 6
Accuracy: 92.560000
Minibaches 7
Accuracy: 92.560000
Minibaches 8
Accuracy: 92.360000
Minibaches 9
Accuracy: 92.513333
chunk 100
Minibaches 0
Accuracy: 92.526667
Minibaches 1
Accuracy: 92.620000
Minibaches 2
Accuracy: 92.526667
Minibaches 3
Accuracy: 92.546667
Minibaches 4
Accuracy: 92.560000
Minibaches 5
Accuracy: 92.480000
Minibaches 6
Accuracy: 92.453333
Minibaches 7
Accuracy: 92.473333
Minibaches 8
Accuracy: 92.346667
Minibaches 9
Accuracy: 92.353333
NO Chunks...
TIME : 1.5554003715515137
Ended 0
----------------------------------------------

chunk 1
Minibaches 0
Accuracy: 92.440000
Minibaches 1
Accuracy: 92.473333
Minibaches 2
Accuracy: 92.506667
Minibaches 3
Accuracy: 92.486667
Minibaches 4
Accuracy: 92.360000
Minibaches 5
Accuracy: 92.313333
Minibaches 6
Accuracy: 92.320000
Minibaches 7
Accuracy: 92.326667
Minibache

Accuracy: 91.793333
Minibaches 8
Accuracy: 91.813333
Minibaches 9
Accuracy: 91.860000
chunk 24
Minibaches 0
Accuracy: 91.886667
Minibaches 1
Accuracy: 91.853333
Minibaches 2
Accuracy: 91.766667
Minibaches 3
Accuracy: 91.800000
Minibaches 4
Accuracy: 91.546667
Minibaches 5
Accuracy: 91.533333
Minibaches 6
Accuracy: 91.500000
Minibaches 7
Accuracy: 91.386667
Minibaches 8
Accuracy: 91.393333
Minibaches 9
Accuracy: 91.400000
chunk 25
Minibaches 0
Accuracy: 91.433333
Minibaches 1
Accuracy: 91.533333
Minibaches 2
Accuracy: 91.613333
Minibaches 3
Accuracy: 91.573333
Minibaches 4
Accuracy: 91.573333
Minibaches 5
Accuracy: 91.586667
Minibaches 6
Accuracy: 91.593333
Minibaches 7
Accuracy: 91.593333
Minibaches 8
Accuracy: 91.660000
Minibaches 9
Accuracy: 91.686667
chunk 26
Minibaches 0
Accuracy: 91.700000
Minibaches 1
Accuracy: 91.700000
Minibaches 2
Accuracy: 91.733333
Minibaches 3
Accuracy: 91.706667
Minibaches 4
Accuracy: 91.706667
Minibaches 5
Accuracy: 91.720000
Minibaches 6
Accuracy: 91.706

Accuracy: 91.886667
Minibaches 6
Accuracy: 92.073333
Minibaches 7
Accuracy: 92.073333
Minibaches 8
Accuracy: 91.980000
Minibaches 9
Accuracy: 91.886667
chunk 49
Minibaches 0
Accuracy: 91.886667
Minibaches 1
Accuracy: 91.813333
Minibaches 2
Accuracy: 91.806667
Minibaches 3
Accuracy: 91.760000
Minibaches 4
Accuracy: 91.813333
Minibaches 5
Accuracy: 91.786667
Minibaches 6
Accuracy: 91.813333
Minibaches 7
Accuracy: 91.766667
Minibaches 8
Accuracy: 91.873333
Minibaches 9
Accuracy: 91.853333
chunk 50
Minibaches 0
Accuracy: 91.866667
Minibaches 1
Accuracy: 91.793333
Minibaches 2
Accuracy: 91.833333
Minibaches 3
Accuracy: 91.806667
Minibaches 4
Accuracy: 91.746667
Minibaches 5
Accuracy: 91.780000
Minibaches 6
Accuracy: 91.913333
Minibaches 7
Accuracy: 91.873333
Minibaches 8
Accuracy: 91.826667
Minibaches 9
Accuracy: 91.886667
chunk 51
Minibaches 0
Accuracy: 91.833333
Minibaches 1
Accuracy: 91.853333
Minibaches 2
Accuracy: 91.773333
Minibaches 3
Accuracy: 91.853333
Minibaches 4
Accuracy: 91.860

Accuracy: 92.273333
chunk 73
Minibaches 0
Accuracy: 92.326667
Minibaches 1
Accuracy: 92.366667
Minibaches 2
Accuracy: 92.440000
Minibaches 3
Accuracy: 92.520000
Minibaches 4
Accuracy: 92.506667
Minibaches 5
Accuracy: 92.600000
Minibaches 6
Accuracy: 92.533333
Minibaches 7
Accuracy: 92.580000
Minibaches 8
Accuracy: 92.573333
Minibaches 9
Accuracy: 92.566667
chunk 74
Minibaches 0
Accuracy: 92.566667
Minibaches 1
Accuracy: 92.593333
Minibaches 2
Accuracy: 92.660000
Minibaches 3
Accuracy: 92.573333
Minibaches 4
Accuracy: 92.566667
Minibaches 5
Accuracy: 92.540000
Minibaches 6
Accuracy: 92.486667
Minibaches 7
Accuracy: 92.453333
Minibaches 8
Accuracy: 92.346667
Minibaches 9
Accuracy: 92.293333
chunk 75
Minibaches 0
Accuracy: 92.320000
Minibaches 1
Accuracy: 92.326667
Minibaches 2
Accuracy: 92.273333
Minibaches 3
Accuracy: 92.226667
Minibaches 4
Accuracy: 92.366667
Minibaches 5
Accuracy: 92.306667
Minibaches 6
Accuracy: 92.286667
Minibaches 7
Accuracy: 92.340000
Minibaches 8
Accuracy: 92.380

Accuracy: 92.566667
Minibaches 2
Accuracy: 92.646667
Minibaches 3
Accuracy: 92.540000
Minibaches 4
Accuracy: 92.540000
Minibaches 5
Accuracy: 92.533333
Minibaches 6
Accuracy: 92.540000
Minibaches 7
Accuracy: 92.466667
Minibaches 8
Accuracy: 92.400000
Minibaches 9
Accuracy: 92.473333
chunk 98
Minibaches 0
Accuracy: 92.486667
Minibaches 1
Accuracy: 92.413333
Minibaches 2
Accuracy: 92.300000
Minibaches 3
Accuracy: 92.360000
Minibaches 4
Accuracy: 92.400000
Minibaches 5
Accuracy: 92.393333
Minibaches 6
Accuracy: 92.426667
Minibaches 7
Accuracy: 92.460000
Minibaches 8
Accuracy: 92.480000
Minibaches 9
Accuracy: 92.480000
chunk 99
Minibaches 0
Accuracy: 92.493333
Minibaches 1
Accuracy: 92.486667
Minibaches 2
Accuracy: 92.486667
Minibaches 3
Accuracy: 92.440000
Minibaches 4
Accuracy: 92.473333
Minibaches 5
Accuracy: 92.620000
Minibaches 6
Accuracy: 92.593333
Minibaches 7
Accuracy: 92.593333
Minibaches 8
Accuracy: 92.493333
Minibaches 9
Accuracy: 92.533333
chunk 100
Minibaches 0
Accuracy: 92.49

In [ ]:
time_centr=np.load('np_arrays/time_centr.npy')
f_acc=np.load('np_arrays/f_acc.npy')
plot_workers([4,8,12,16,20,24,28,32],time_centr,f_acc)
plot_speedup([4,8,12,16,20,24,28,32],time_centr,f_acc)

In [ ]:
from IPython.display import Audio, display
display(Audio(url='http://peal.io/download/5e289.wav', autoplay=True))
# http://peal.io/download/mr5xx.wav

In [ ]:
# Acc_minibatch=[]
# Rounds_minibatch=[]
# Time_minibatch=[]
# e=0.2
# for i in minibatches :
#     minibatch=i
#     Acc,n_rounds,time_stamps=main(client,w,new,dataset_params,e,chunks,minibatch)
#     new="no"
#     Acc_minibatch.append(Acc)
#     Rounds_minibatch.append(n_rounds)
#     Time_minibatch.append(time_stamps)
#     time.sleep(5)

In [ ]:
# name="Plots/Minibatch_4x_w_"+str(len(w))+"_samples_"+str(dataset_params["n_samples"])+"_features_"+str(dataset_params["n_features"])+"_minib_"+str(minibatches)+".png"
# plot(l1,l2,Acc_minibatch,Rounds_minibatch,Time_minibatch,minibatches,labels,name,'minibatches')

### Save results

In [ ]:
# f = open("result.txt", "a")
# f.writelines(["\n\n==========================================================================="])
# f.writelines(["\nNEW EXPERIMENT:\nparams:",str(dataset_params),"\nminibaches_size:"
#               ,str(minibatches),"\ndifferent e's:",str(e_array),"\nn_workers: ",str(len(w))])
# f.writelines(["\n___________________________________________________________________________"])
# f.writelines(["\nRESULTS:\nAcc_threshold",str(Acc_threshold),"\nRounds_threshold",str(Rounds_threshold),
#              "\nTime_threshold",str(Time_threshold),"\nlen_real1",str(len_real1),"\nAcc_real1",str(Acc_real1)])
# f.close()

### Shutdown client

In [ ]:
client.close()
client.shutdown()